In [1]:
import os

from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
import chromadb

current_dir = os.path.dirname(os.path.abspath("__file__"))
persistent_directory = os.path.join(current_dir, "db", "chroma_db_v3")
client = chromadb.PersistentClient(path=persistent_directory)

embeddings = HuggingFaceEmbeddings(
            model_name="BAAI/bge-m3"
)

db = Chroma(
    client = client,
    embedding_function=embeddings,
    collection_name="smartphone_manual"
)

retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

query = "如何設定指紋辨識?"

relevant_docs = retriever.invoke(query)
for doc in relevant_docs:
    print(doc.page_content)
    print("=" * 60)

Exception ignored in: <function tqdm.__del__ at 0x0000023F31751AB0>
Traceback (most recent call last):
  File "d:\GitHub\Lang_Chain\.venv\lib\site-packages\tqdm\std.py", line 1148, in __del__
    self.close()
  File "d:\GitHub\Lang_Chain\.venv\lib\site-packages\tqdm\notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm' object has no attribute 'disp'


臉部辨識設定:
1. 設定 > 安全性 > 臉部辨識
2. 輸入備用密碼
3. 保持正常距離面對前置相機
4. 緩慢轉動頭部進行掃描
5. 完成註冊

使用建議:
- 同時設定指紋與臉部辨識增加便利性
- 務必設定備用密碼
- 戴口罩時可能影響臉部辨識

7.2 應用程式權限
管理應用程式權限:
設定 > 隱私權 > 權限管理員

可管理的權限:
- 位置(GPS)
- 相機
- 麥克風
- 聯絡人
- 行事曆
- 儲存空間
- 簡訊
- 電話

權限等級:
- 永遠允許
- 僅在使用時允許
- 每次詢問
- 拒絕

建議:
僅授予應用程式必要的權限,定期檢查並移除不必要的權限

7.3 資料備份
雲端備份設定:
設定 > 系統 > 備份

可備份項目:
- 應用程式資料
- 通話記錄
- 聯絡人
- 裝置設定
- 簡訊
- 相片與影片(需額外設定)

備份頻率:
每日自動備份(連接 Wi-Fi 且充電時)
臉部辨識設定:
1. 設定 > 安全性 > 臉部辨識
2. 輸入備用密碼
3. 保持正常距離面對前置相機
4. 緩慢轉動頭部進行掃描
5. 完成註冊

使用建議:
- 同時設定指紋與臉部辨識增加便利性
- 務必設定備用密碼
- 戴口罩時可能影響臉部辨識

7.2 應用程式權限
管理應用程式權限:
設定 > 隱私權 > 權限管理員

可管理的權限:
- 位置(GPS)
- 相機
- 麥克風
- 聯絡人
- 行事曆
- 儲存空間
- 簡訊
- 電話

權限等級:
- 永遠允許
- 僅在使用時允許
- 每次詢問
- 拒絕

建議:
僅授予應用程式必要的權限,定期檢查並移除不必要的權限

7.3 資料備份
雲端備份設定:
設定 > 系統 > 備份

可備份項目:
- 應用程式資料
- 通話記錄
- 聯絡人
- 裝置設定
- 簡訊
- 相片與影片(需額外設定)

備份頻率:
每日自動備份(連接 Wi-Fi 且充電時)
音訊規格:
- 立體聲雙喇叭
- 支援 Dolby Atmos 環繞音效
- Hi-Res Audio 高解析音訊
- 支援藍牙編解碼器:SBC/AAC/aptX/LDAC

6.3 通知管理
設定 > 通知

管理應用程式通知:
1. 選擇應用程式
2. 開啟/關閉通知
3. 設定通知類別
4. 通知顯示方式(橫幅/鎖定螢幕/通知中心)
5. 通知聲音與震

In [2]:
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_ollama import ChatOllama
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
import chromadb

current_dir = os.path.dirname(os.path.abspath("__file__"))
persistent_directory = os.path.join(current_dir, "db", "chroma_db_v3")
client = chromadb.PersistentClient(path=persistent_directory)

embeddings = HuggingFaceEmbeddings(
            model_name="BAAI/bge-m3"
)

db = Chroma(
    client = client,
    embedding_function=embeddings,
    collection_name="smartphone_manual"
)



def retriever_docs(question):
    retriever = db.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 5}
    )
    relevant_docs = retriever.invoke(question)

    
    content = "\n\n".join([doc.page_content for doc in relevant_docs])

    
    return {
        "context":content,
        "question":question,
        }

template = """你是一個智慧型手機的客服助手。請根據以下參考資料回答使用者的問題。

參考資料：
{context}

使用者問題：{question}

請用繁體中文回答，並且：
1. 只根據參考資料回答，不要編造內容
2. 如果參考資料中沒有答案，請誠實說「我在資料中找不到相關資訊」
3. 回答要清楚、具體、有條理

回答："""

prompt = ChatPromptTemplate.from_template(template)
llm = ChatOllama(model="gpt-oss:20b")

rag_chain = (
    RunnableLambda(retriever_docs)
    | prompt
    | llm
    | StrOutputParser()
)

result = rag_chain.invoke("藍牙怎麼使用?")


print(result)

Exception ignored in: <function tqdm.__del__ at 0x0000020C3730DF30>
Traceback (most recent call last):
  File "d:\GitHub\Lang_Chain\.venv\lib\site-packages\tqdm\std.py", line 1148, in __del__
    self.close()
  File "d:\GitHub\Lang_Chain\.venv\lib\site-packages\tqdm\notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm' object has no attribute 'disp'


藍牙使用說明（僅根據參考資料）  

1. **開啟藍牙**  
   - 進入「設定」>「藍牙」  
   - 開啟藍牙開關  

2. **將要配對的裝置設定為配對模式**  
   - 例如無線耳機、智慧手錶、藍牙鍵盤/滑鼠、車用藍牙系統、或支援藍牙傳檔的裝置，請先將該裝置設定為可被發現的配對模式（具體操作視裝置說明書而定）。  

3. **在手機上選擇裝置**  
   - 在「可用裝置」列表中點選剛才設定為配對模式的裝置。  

4. **確認配對碼（如需要）**  
   - 若系統要求輸入或確認配對碼，請照提示輸入或確認，完成配對。  

5. **使用**  
   - 配對成功後，裝置會顯示為已連線，可即時使用（例如播放音樂、輸入文字、傳輸檔案等）。  

**支援的藍牙裝置類型**  
- 無線耳機/喇叭  
- 智慧手錶/手環  
- 藍牙鍵盤/滑鼠  
- 車用藍牙系統  
- 檔案傳輸（藍牙傳檔）  

**藍牙規格**  
- 版本：藍牙 5.3  
- 傳輸距離：最遠 10 公尺（無障礙）  
- 支援多裝置同時連線  

如果您需要進一步的設定（例如藍牙音量、資料傳輸方式等），請參考裝置說明書或系統設定。
